In [ ]:
#!pip install mne
#!pip install giotto-tda

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import sys
import os
from glob import glob
import numpy as np
import scipy as sp
import statistics, math
import random as rd
import pandas as pd
#import dataframe_image as dfi
pd.options.display.float_format = '{:,.2f}'.format

from math import sqrt, ceil

from sklearn import metrics
from sklearn import decomposition, cluster
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from scipy.spatial.distance import correlation
from IPython.display import display

In [ ]:
import mne
#import mne_connectivity

from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap, annotate_muscle_zscore, peak_finder,
                               find_eog_events, find_ecg_events, ica_find_ecg_events, find_bad_channels_maxwell)
from mne.time_frequency import (tfr_array_multitaper, psd_array_multitaper, csd_array_multitaper, csd_array_morlet)

from mne_connectivity import seed_target_indices, spectral_connectivity_epochs, spectral_connectivity_time

print(mne.__file__)
print(mne.__version__)

/usr/local/lib/python3.7/dist-packages/mne/__init__.py
1.1.1


In [ ]:
import plotly.graph_objects as go
from gtda.time_series import SingleTakensEmbedding, TakensEmbedding, takens_embedding_optimal_parameters
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import BettiCurve, Scaler, Filtering, Amplitude, PersistenceEntropy, NumberOfPoints
from gtda.plotting import plot_point_cloud


In [ ]:
# Configure the graphics engine

import matplotlib.pyplot as plt
plt.rc('figure', max_open_warning=200)
%matplotlib notebook
#%matplotlib qt
#%matplotlib inline

#from google.colab import files

# Loading data

In [ ]:
ECG_ch = ['ecg']
resp_ch = ['resp']
sound_ch = ['Sound']
accel_ch = ['Accelerometer']

data_raw = mne.io.read_raw_edf(os.path.join(root_path, r"m10_40.edf"), preload=True)
sampling_rate = data_raw.info['sfreq']

print(data_raw.info.ch_names)
print(data_raw.info)


#data_eog = data.copy().pick_channels(EOG_ch)
data_ecg = data_raw.copy().pick_channels(ECG_ch)
#data_ecg.plot(scalings = dict(eeg=100e-4), duration=10)
data_resp = data_raw.copy().pick_channels(resp_ch)
#data_resp.plot(scalings = dict(eeg=100e-3), duration=10)

data_raw.drop_channels(['ecg', 'resp', 'Sound', 'Accelerometer'])
#data_raw.rename_channels(lambda s: s[:-1].upper()+s[-1])
data_raw.rename_channels({'Ft7':'FT7', 'Fc3':'FC3', 'Fcz':'FCz', 'Fc4':'FC4', 'Ft8':'FT8', 'Tp7':'TP7', 'Cp3':'CP3', 
                          'Cpz':'CPz', 'Cp4':'CP4', 'Tp8':'TP8', 'Po3':'PO3', 'Poz':'POz', 'Po4':'PO4', 'Po7':'PO7', 
                          'Po8':'PO8'})
data_raw.set_montage('standard_1020')
#data.set_channel_types(change_types)
print(data_raw.ch_names)
#data_raw.plot(duration=10)
print()
data_raw

Extracting EDF parameters from /content/gdrive/My Drive/Colab Projects/Tibet/m10/m10_40.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 491999  =      0.000 ...   983.998 secs...
['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'Ft7', 'Fc3', 'Fcz', 'Fc4', 'Ft8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'Tp7', 'Cp3', 'Cpz', 'Cp4', 'Tp8', 'A1', 'T5', 'P3', 'Pz', 'P4', 'T6', 'A2', 'P5', 'Po3', 'Poz', 'Po4', 'P6', 'Po7', 'O1', 'Oz', 'O2', 'Po8', 'ecg', 'Sound', 'Accelerometer', 'resp']
<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, F7, F3, Fz, F4, F8, Ft7, Fc3, Fcz, Fc4, Ft8, T3, ...
 chs: 44 EEG
 custom_ref_applied: False
 highpass: 0.5 Hz
 lowpass: 70.0 Hz
 meas_date: 2020-03-01 06:31:33 UTC
 nchan: 44
 projs: []
 sfreq: 500.0 Hz
>
['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'A1', 'T5', 'P3', 'Pz', 'P4', 'T6', 'A2', 'P

<RawEDF | m10_40.edf, 40 x 492000 (984.0 s), ~150.2 MB, data loaded>

# Preprocessing EEG data



In [ ]:
# Re-referencing raw data (common average reference)

bands = [(0.9, 4, 'Delta (0.9-4 Hz)'), (4, 8, 'Theta (4-8 Hz)'), (8, 14, 'Alpha (8-14 Hz)'), 
         (14, 25, 'Beta (14-25 Hz)'), (25, 40, 'Gamma (25-40 Hz)')]

data_raw_filt = data_raw.copy().filter(l_freq=bands[0][0], h_freq=bands[-1][1])
#fig_orig = data_raw_filt.plot(duration=20)

rr_data_raw, ref_ch = mne.set_eeg_reference(data_raw, ref_channels='average', copy=True, projection=False, ch_type='auto')
rr_data_raw_filt = rr_data_raw.copy().filter(l_freq=bands[0][0], h_freq=bands[-1][1])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1835 samples (3.670 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.1s finished


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1835 samples (3.670 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.8s finished


# Applying ICA

In [ ]:
# Re-referenced data
rr_data_ica = rr_data_raw.copy()
#rr_data_ica.info['bads']=["F4","F8"]
#rr_data_ica.interpolate_bads(reset_bads=True)

rr_data_ica.filter(l_freq=1., h_freq=None)

rr_ica = ICA(n_components=.99, max_iter='auto', random_state=97)
rr_ica.fit(rr_data_ica)


Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Filter length: 825 samples (1.650 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


Fitting ICA to data using 40 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.3s finished


Selecting by explained variance: 27 components
Fitting ICA took 22.7s.


Method,fastica
Fit,26 iterations on raw data (467501 samples)
ICA components,27
Explained variance,99.1 %
Available PCA components,40
Channel types,eeg
ICA components marked for exclusion,—


In [ ]:
# Exclude ECG channel (re-referenced data)
rr_ica.exclude = [0, 3] # EOG/ECG components, indexes depending on data (individual)
reref_data = rr_data_raw.copy()
rr_ica.apply(reref_data)

#reref_data.plot()
#reref_data.plot_psd()

#rr_data_notch = reref_data.copy().notch_filter([50])
#rr_data_notch.plot_psd()#fmin=0, fmax=105)
print()

Applying ICA to Raw instance
    Transforming to ICA space (27 components)
    Zeroing out 1 ICA component
    Projecting back using 40 PCA components



# Epoching EEG data

In [ ]:
# Filtering raw_data
bands = [(0.9, 4, 'Delta (0.9-4 Hz)', 'D'), (4, 8, 'Theta (4-8 Hz)', 'T'), (8, 14, 'Alpha (8-14 Hz)', 'A'), 
         (14, 25, 'Beta (14-25 Hz)', 'B'), (25, 40, 'Gamma (25-40 Hz)', 'G')]

str_freq = [bands[i][3] for i in range(len(bands))]
#str_freq_rr = ['r'+_str for _str in str_freq]
str_freq_rr = str_freq

# Re-referenced data
data_rr_filt = reref_data.copy().filter(l_freq=bands[0][0], h_freq=bands[-1][1])
data_rr_filt_spectr = []
for i in range(len(bands)):
    data_rr_filt_spectr.append(reref_data.copy().filter(l_freq=bands[i][0], h_freq=bands[i][1]))

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1835 samples (3.670 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.9 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.90
- Lower transition bandwidth: 0.90 Hz (-6 dB cutoff frequency: 0.45 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1835 samples (3.670 sec)



[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 825 samples (1.650 sec)



...

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 14 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 14.00
- Lower transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 12.25 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 473 samples (0.946 sec)



[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 265 samples (0.530 sec)



[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.0s finished


In [ ]:
# Epochs by fixed length events, duration 1s

fix_1s_events_rr = mne.make_fixed_length_events(data_rr_filt, start=0.5, duration=1.)#, overlap=0.2)
kwargs = dict(baseline=None, tmin=-0.5, tmax=0.5, preload=True)

epochs_fix_1s_rr = mne.Epochs(data_rr_filt, fix_1s_events_rr.astype(int), **kwargs)
epochs_fix_1s_rr = epochs_fix_1s_rr[2:-2] #remove first and last 2sec due to possible filtering artefacts

print(fix_1s_events_rr.shape)
print(epochs_fix_1s_rr)
print(epochs_fix_1s_rr.get_data().shape)
#print(epochs_fix_1s_rr.events)

epochs_fix_rr_spectr = []
for i in range(len(bands)):   
    epochs_fix_rr_spectr.append(mne.Epochs(data_rr_filt_spectr[i], fix_1s_events_rr.astype(int), **kwargs))
    epochs_fix_rr_spectr[i] = epochs_fix_rr_spectr[i][2:-2] 


Not setting metadata
1167 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1167 events and 501 original time points ...
0 bad epochs dropped
(1167, 3)
<Epochs |  1163 events (all good), -0.5 - 0.5 sec, baseline off, ~177.9 MB, data loaded,
 '1': 1163>
(1163, 40, 501)
Not setting metadata
1167 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1167 events and 501 original time points ...
0 bad epochs dropped
Not setting metadata
1167 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1167 events and 501 original time points ...
0 bad epochs dropped
Not setting metadata
1167 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1167 events and 501 original time points ...
0 bad epochs dropped
Not setting metadata
1167 matching events

In [ ]:
# Global variables
regions = [(['Fp1','Fp2','Fpz'], 'Fp', 'Pre-frontal'), (['F7','F3','FT7','FC3'], 'LF', 'Left Frontal'), 
           (['Fz','FCz'], 'MF', 'Midline Frontal'), (['F4','F8','FC4','FT8'], 'RF', 'Right Frontal'),
           (['T3','TP7','T5'], 'LT', 'Left Temporal'), (['T4','TP8','T6'], 'RT', 'Right Temporal'), 
           (['C3','CP3'], 'LC', 'Left Central'), (['Cz','CPz'], 'MC', 'Midline Central'), (['C4','CP4'], 'RC', 'Right Central'),
           (['P3','P5'], 'LP', 'Left Parietal'), (['Pz'], 'MP', 'Midline Parietal'), (['P4','P6'], 'RP', 'Right Parietal'), 
           (['PO3','PO7','O1'], 'LO', 'Left Occipital'), (['POz','Oz'], 'MO', 'Midline Occipital'), 
           (['PO4','PO8','O2'], 'RO', 'Right Occipital')]
regions_plt = [(['Fp1','Fp2','Fpz'], 'Fp', 'Pre-frontal'), (['F7','F3','FT7','FC3'], 'LF', 'Left Frontal'), 
           (['Fz','FCz'], 'MF', 'Midline Frontal'), (['F4','F8','FC4','FT8'], 'RF', 'Right Frontal'),
           (['T3','TP7','T5'], 'LT', 'Left Temporal'), (['T4','TP8','T6'], 'RT', 'Right Temporal'), 
           (['C3','CP3','Cz','CPz','C4','CP4'], 'Cen', 'Central'), (['P3','P5','Pz','P4','P6'], 'Par', 'Parietal'), 
           (['PO3','PO7','O1','POz','Oz','PO4','PO8','O2'], 'Occ', 'Occipital')]


ch_names = data_raw.ch_names
n_freq = len(str_freq_rr)
n_channels = len(ch_names)
n_regions = len(regions)
n_regions_plt = len(regions_plt)

n_samples = epochs_fix_1s_rr.__len__()
n_times = len(epochs_fix_1s_rr.get_data()[0,0,:])

print(ch_names)

loc_masks = [[ch_names[i] in reg for i in range(n_channels)] for (reg, _, _) in regions]

['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'A1', 'T5', 'P3', 'Pz', 'P4', 'T6', 'A2', 'P5', 'PO3', 'POz', 'PO4', 'P6', 'PO7', 'O1', 'Oz', 'O2', 'PO8']


In [ ]:
# Averaging epochs by region

# Re-referenced data
ft_rr_epochs = epochs_fix_1s_rr.get_data()
ft_rr_epochs_loc = np.array([np.mean(ft_rr_epochs[:,_loc_mask,:], axis=1) for _loc_mask in loc_masks]).transpose(1,0,2)

print(ft_rr_epochs.shape)
print(ft_rr_epochs_loc.shape)
print(ch_names)

(1163, 40, 501)
(1163, 15, 501)
['Fp1', 'Fpz', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'A1', 'T5', 'P3', 'Pz', 'P4', 'T6', 'A2', 'P5', 'PO3', 'POz', 'PO4', 'P6', 'PO7', 'O1', 'Oz', 'O2', 'PO8']


# TDA features

In [ ]:
# Global variables

#tda_root_path = root_path + 'R3/' # embedding dimension R3
tda_root_path = root_path + 'R2/' # embedding dimension R2

tda_bands = [(0.9, 4, 'D', 'Delta (0.9-4 Hz)', 15), (4, 8, 'T', 'Theta (4-8 Hz)', 12), (8, 14, 'A', 'Alpha (8-14 Hz)', 10), 
             (14, 25, 'B', 'Beta (14-25 Hz)', 7), (25, 40, 'G', 'Gamma (25-40 Hz)', 4)]

tda_metrics = [('bottleneck','botl'), ('wasserstein', 'wass'), ('betti', 'bet'), ('landscape', 'land')] 
#           ('silhouette', 'silh'), ('heat', 'heat'), ('persistence_image', 'img')]

#hom_dims = [1, 2] # Considered homology dimensions
hom_dims = [1] # Considered homology dimensions

#l_norms = [1,2] # Norm orders for produsing features
l_norms = [2] # Norm orders for produsing features

persistence = VietorisRipsPersistence(
    homology_dimensions=hom_dims, 
    n_jobs=-1,
)

In [ ]:
# Time Series data from Epochs by tda spectrum

# Re-referenced data
ts_ft_rr_tda_sp = []
ts_ft_rr_tda_loc_sp = []
pcl_ft_rr_tda_sp = []
pcl_ft_rr_tda_loc_sp = []

ts_ft_rr_tda = epochs_filt_rr.get_data()
ts_ft_rr_tda_loc = np.array([np.mean(ts_ft_rr_tda[:,_loc_mask,:], axis=1) for _loc_mask in loc_masks]).transpose(1,0,2)
print(ts_ft_rr_tda_loc.shape)

for _freq in range(n_freq):   
    ts_ft_rr_tda_sp.append(epochs_filt_rr_spectr[_freq].get_data())
    ts_ft_rr_tda_loc_sp.append(np.array([np.mean(ts_ft_rr_tda_sp[_freq][:,_loc_mask,:], axis=1) for _loc_mask in loc_masks]).transpose(1,0,2))
    print(ts_ft_rr_tda_loc_sp[_freq].shape)


(1046, 15, 501)
(1046, 15, 501)
(1046, 15, 501)
(1046, 15, 501)
(1046, 15, 501)
(1046, 15, 501)


In [ ]:
# Converting time series to point clouds (attractor)

# Build attractor
emb_time_del = 12 # embedding time delay

#emb_dim = 3 # embedding dimension R3
emb_dim = 2 # embedding dimension R2

stride = 2 # downsampling

attr = TakensEmbedding(
    time_delay=emb_time_del,
    dimension=emb_dim,
    flatten=False,
    stride=stride,
    #n_jobs=2,
) # Defining attractor

# Re-referenced
pcl_ft_rr_tda = attr.fit_transform(ts_ft_rr_tda)
pcl_ft_rr_tda_loc = attr.fit_transform(ts_ft_rr_tda_loc)
#attr.fit_transform_plot(ts_ft_rr_tda[:,0,:], sample=100)

for _freq in range(n_freq):
    attr_sp = TakensEmbedding(
    time_delay=tda_bands[_freq][4],
    dimension=emb_dim,
    flatten=False,
    stride=stride,
    #n_jobs=2,
    ) # Defining attractor
  
    pcl_ft_rr_tda_sp.append(attr_sp.fit_transform(ts_ft_rr_tda_sp[_freq]))
    pcl_ft_rr_tda_loc_sp.append(attr_sp.fit_transform(ts_ft_rr_tda_loc_sp[_freq]))



In [ ]:
# Persistent diagrams (full range, re-referenced data, averaged by region)
dir_name = 'pd_ft_rr_tda_loc'
target_dir = tda_root_path + dir_name
if not os.path.exists(target_dir):
    os.mkdir(target_dir)

pd_ft_rr_tda_loc_list = []
for i in range(n_regions):
    if not os.path.exists(os.path.join(target_dir, dir_name+'_'+str(i)+'.npy')):
        pd_ft_rr_tda_loc_list.append(persistence.fit_transform(pcl_ft_rr_tda_loc[:,i,:,:]))
        np.save(os.path.join(target_dir, dir_name+'_'+str(i)+'.npy'), pd_ft_rr_tda_loc_list[i])
    else:
        pd_ft_rr_tda_loc_list.append(np.load(os.path.join(target_dir, dir_name+'_'+str(i)+'.npy')))
    print(i, pd_ft_rr_tda_loc_list[i].shape)

0 (1046, 74, 3)
1 (1046, 71, 3)
2 (1046, 74, 3)
3 (1046, 70, 3)
4 (1046, 71, 3)
5 (1046, 71, 3)
6 (1046, 72, 3)
7 (1046, 73, 3)
8 (1046, 70, 3)
9 (1046, 70, 3)
10 (1046, 71, 3)
11 (1046, 78, 3)
12 (1046, 71, 3)
13 (1046, 74, 3)
14 (1046, 71, 3)


In [ ]:
# Persistent diagrams (by TDA spectrum, re-referenced data, averaged by region)

for _freq in range(len(tda_bands)):
    dir_name = 'pd_ft_rr_tda_loc_'+tda_bands[_freq][2]
    target_dir = tda_root_path + dir_name
    if not os.path.exists(target_dir):
        os.mkdir(target_dir)

    pd_ft_rr_tda_loc_list = []
    print(tda_bands[_freq][3])
    for i in range(n_regions):
        if not os.path.exists(os.path.join(target_dir, dir_name+'_'+str(i)+'.npy')):
            pd_ft_rr_tda_loc_list.append(persistence.fit_transform(pcl_ft_rr_tda_loc_sp[_freq][:,i,:,:]))
            np.save(os.path.join(target_dir, dir_name+'_'+str(i)+'.npy'), pd_ft_rr_tda_loc_list[i])
        else:
            pd_ft_rr_tda_loc_list.append(np.load(os.path.join(target_dir, dir_name+'_'+str(i)+'.npy')))
        print(i, pd_ft_rr_tda_loc_list[i].shape)

Delta (0.9-4 Hz)
0 (1046, 76, 3)
1 (1046, 68, 3)
2 (1046, 81, 3)
3 (1046, 77, 3)
4 (1046, 73, 3)
5 (1046, 73, 3)
6 (1046, 69, 3)
7 (1046, 78, 3)
8 (1046, 81, 3)
9 (1046, 75, 3)
10 (1046, 73, 3)
11 (1046, 70, 3)
12 (1046, 73, 3)
13 (1046, 84, 3)
14 (1046, 66, 3)
Theta (4-8 Hz)
0 (1046, 112, 3)
1 (1046, 117, 3)
2 (1046, 108, 3)
3 (1046, 111, 3)
4 (1046, 106, 3)
5 (1046, 123, 3)
6 (1046, 109, 3)
7 (1046, 114, 3)
8 (1046, 104, 3)
9 (1046, 110, 3)
10 (1046, 126, 3)
11 (1046, 124, 3)
12 (1046, 118, 3)
13 (1046, 125, 3)
14 (1046, 122, 3)
Alpha (8-14 Hz)
0 (1046, 116, 3)
1 (1046, 115, 3)
2 (1046, 111, 3)
3 (1046, 110, 3)
4 (1046, 111, 3)
5 (1046, 107, 3)
6 (1046, 106, 3)
7 (1046, 108, 3)
8 (1046, 102, 3)
9 (1046, 104, 3)
10 (1046, 111, 3)
11 (1046, 121, 3)
12 (1046, 110, 3)
13 (1046, 111, 3)
14 (1046, 121, 3)
Beta (14-25 Hz)
0 (1046, 77, 3)
1 (1046, 76, 3)
2 (1046, 78, 3)
3 (1046, 77, 3)
4 (1046, 79, 3)
5 (1046, 77, 3)
6 (1046, 80, 3)
7 (1046, 77, 3)
8 (1046, 78, 3)
9 (1046, 78, 3)
10 (1046, 7

In [ ]:
pcl_epochs_fix_1s_ovl_smpl = pcl_epochs_fix_1s_ovl_alpha[100,5,:,:]
pcl_epochs_fix_1s_ovl_smpl = pcl_epochs_fix_1s_ovl_smpl[None, :, :]
print(pcl_epochs_fix_1s_ovl_smpl.shape)

pd_fix_1s_ovl_full = persistence.fit_transform(pcl_epochs_fix_1s_ovl_smpl)

print("Persistence diagram for signal")
persistence.plot(pd_fix_1s_ovl_full)

(1, 241, 3)
Persistence diagram for signal


In [ ]:
print(pd_fix_1s_ovl_full.shape)

(1, 82, 3)


In [ ]:
print(pd_fix_1s_ovl_full.shape)

persist_filt = Filtering(
    homology_dimensions=[1,2],
    epsilon=0.0000001,
)
pd_fix_1s_ovl_filt = persist_filt.fit_transform(pd_fix_1s_ovl_full)

print(pd_fix_1s_ovl_filt.shape)

print("Persistence diagram for signal")
persistence.plot(pd_fix_1s_ovl_filt, sample=0)

(1, 107, 3)
(1, 40, 3)
Persistence diagram for signal


In [ ]:
pcl_epochs_fix_1s_ovl_smpl = pcl_epochs_fix_1s_ovl_theta[100,0,:,:]
pcl_epochs_fix_1s_ovl_smpl = pcl_epochs_fix_1s_ovl_smpl[None, :, :]
print(pcl_epochs_fix_1s_ovl_smpl.shape)

pd_fix_1s_ovl_full = persistence.fit_transform(pcl_epochs_fix_1s_ovl_smpl)

print("Persistence diagram for signal")
persistence.plot(pd_fix_1s_ovl_full)

(1, 239, 3)
Persistence diagram for signal


In [ ]:
persist_filt = Filtering(
    homology_dimensions=[1,2],
    epsilon=0.0000001,
)

pd_fix_1s_ovl_filt = persist_filt.fit_transform(pd_fix_1s_ovl_full)

print(pd_fix_1s_ovl_filt.shape)

print("Persistence diagram for signal")
persistence.plot(pd_fix_1s_ovl_filt, sample=0)

(1, 14, 3)
Persistence diagram for signal


In [ ]:
m10_sample_ts = epochs_fix_1s_ovl.get_data(picks=0, item=100).reshape(epochs_fix_1s_ovl.get_data().shape[2])
print(m10_sample_ts.shape)

(501,)


In [ ]:
embedding_dimension = 3
embedding_time_delay = 8
stride = 3

embedder_sample = SingleTakensEmbedding(
    parameters_type="fixed",
#    n_jobs=2,
    time_delay=embedding_time_delay,
    dimension=embedding_dimension,
    stride=stride,
)

In [ ]:
m10_sample_point_cl = embedder_sample.fit_transform(m10_sample_ts)
print(f"Shape of embedded time series: {m10_sample_point_cl.shape}")
plot_point_cloud(m10_sample_point_cl)

Shape of embedded time series: (162, 3)


In [ ]:
# Change array type for next step
print(m10_sample_point_cl.shape)
m10_sample_point_cl = m10_sample_point_cl[None, :, :]
print(m10_sample_point_cl.shape)

(162, 3)
(1, 162, 3)


In [ ]:
# Build persistent diagram
homology_dimensions = [0, 1, 2]

persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)

m10_sample_per_diag = persistence.fit_transform(m10_sample_point_cl)

print("Persistence diagram for signal")
persistence.fit_transform_plot(m10_sample_point_cl)

Persistence diagram for signal


array([[[0.00000000e+00, 2.74064263e-07, 0.00000000e+00],
        [0.00000000e+00, 3.46944972e-07, 0.00000000e+00],
        [0.00000000e+00, 3.65284336e-07, 0.00000000e+00],
        [0.00000000e+00, 6.63441142e-07, 0.00000000e+00],
        [0.00000000e+00, 6.86488420e-07, 0.00000000e+00],
        [0.00000000e+00, 7.15882379e-07, 0.00000000e+00],
        [0.00000000e+00, 7.32955300e-07, 0.00000000e+00],
        [0.00000000e+00, 8.02591615e-07, 0.00000000e+00],
        [0.00000000e+00, 8.31667705e-07, 0.00000000e+00],
        [0.00000000e+00, 9.12478470e-07, 0.00000000e+00],
        [0.00000000e+00, 9.29409225e-07, 0.00000000e+00],
        [0.00000000e+00, 9.41982591e-07, 0.00000000e+00],
        [0.00000000e+00, 9.47148635e-07, 0.00000000e+00],
        [0.00000000e+00, 9.48348088e-07, 0.00000000e+00],
        [0.00000000e+00, 1.00823399e-06, 0.00000000e+00],
        [0.00000000e+00, 1.01375474e-06, 0.00000000e+00],
        [0.00000000e+00, 1.01938326e-06, 0.00000000e+00],
        [0.000

# Testing TDA methods on single epoch of 1 channel

In [ ]:
m10_sample_ts = epochs.get_data(picks=0, item=0).reshape(epochs.get_data().shape[2])
time_delay, dimension = takens_embedding_optimal_parameters(m10_sample_ts,10, 10)
print('time_delay = ', time_delay)
print('dimension = ', dimension)

time_delay =  3
dimension =  5


In [ ]:
embedding_dimension = 3
embedding_time_delay = 8
stride = 3

embedder_sample = SingleTakensEmbedding(
    parameters_type="fixed",
#    n_jobs=2,
    time_delay=embedding_time_delay,
    dimension=embedding_dimension,
    stride=stride,
)

In [ ]:
m10_sample_point_cl = embedder_sample.fit_transform(m10_sample_ts)
print(f"Shape of embedded time series: {m10_sample_point_cl.shape}")
plot_point_cloud(m10_sample_point_cl)

Shape of embedded time series: (128, 3)


In [ ]:
# Change array type for next step
print(m10_sample_point_cl.shape)
m10_sample_point_cl = m10_sample_point_cl[None, :, :]
print(m10_sample_point_cl.shape)


(128, 3)
(1, 128, 3)


In [ ]:
# Build persistent diagram
homology_dimensions = [0, 1, 2]

persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)

m10_sample_per_diag = persistence.fit_transform(m10_sample_point_cl)

print("Persistence diagram for signal")
persistence.fit_transform_plot(m10_sample_point_cl)

Persistence diagram for signal


array([[[0.00000000e+00, 1.50544076e-06, 0.00000000e+00],
        [0.00000000e+00, 1.67911276e-06, 0.00000000e+00],
        [0.00000000e+00, 1.88592207e-06, 0.00000000e+00],
        [0.00000000e+00, 2.26344946e-06, 0.00000000e+00],
        [0.00000000e+00, 2.34499703e-06, 0.00000000e+00],
        [0.00000000e+00, 2.70918122e-06, 0.00000000e+00],
        [0.00000000e+00, 2.79882806e-06, 0.00000000e+00],
        [0.00000000e+00, 2.83279496e-06, 0.00000000e+00],
        [0.00000000e+00, 2.90430148e-06, 0.00000000e+00],
        [0.00000000e+00, 3.33883395e-06, 0.00000000e+00],
        [0.00000000e+00, 3.38988389e-06, 0.00000000e+00],
        [0.00000000e+00, 3.42249382e-06, 0.00000000e+00],
        [0.00000000e+00, 3.45864419e-06, 0.00000000e+00],
        [0.00000000e+00, 3.47875812e-06, 0.00000000e+00],
        [0.00000000e+00, 3.48215895e-06, 0.00000000e+00],
        [0.00000000e+00, 3.52137272e-06, 0.00000000e+00],
        [0.00000000e+00, 3.72336353e-06, 0.00000000e+00],
        [0.000

In [ ]:
# Getting Betti Sequence
betti_sequence = BettiCurve()

m10_sample_betti_seq = betti_sequence.fit_transform(m10_sample_per_diag)
betti_sequence.fit_transform_plot(m10_sample_per_diag)


array([[[127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127,
         126, 126, 125, 124, 124, 124, 123, 122, 122, 122, 120, 118,
         118, 118, 118, 115, 111, 111, 109, 107, 102,  98,  97,  97,
          95,  94,  91,  86,  82,  78,  73,  64,  60,  59,  53,  51,
          50,  47,  46,  42,  40,  39,  37,  34,  32,  30,  25,  23,
          21,  20,  17,  15,  15,  14,  14,  14,  12,  11,   9,   9,
           9,   8,   8,   8,   8,   7,   7,   7,   6,   6,   5,   4,
           4,   4,   4,   4,   4,   4,   3,   3,   3,   3,   3,   3,
           2,   1,   1,   0],
        [  1,   0,   0,   1,   1,   3,   3,   3,   4,   3,   4,   4,
           3,   3,   3,   4,   4,   5,   7,  10,   9,   8,  10,  10,
          11,  11,  14,  14,  15,  15,  14,  14,  14,  14,  14,  14,
          12,   9,  10,  10,   9,   7,   8,   8,   8,   9,   8,   8,
           8,   7,   6,   7,   8,   6,   6,   4,   3,   3,   3,   3,
           3,   3,   4,   2,   3,   3,   3,   3,   3,   2,   1,   1,
    

In [ ]:
persist_SR = SparseRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=6
)

m10_sample_per_diag = persist_SR.fit_transform(m10_sample_point_cl)

print("Persistence diagram for signal")
persist_SR.fit_transform_plot(m10_sample_point_cl)

Persistence diagram for signal


array([[[0.00000000e+00, 1.26098164e-05, 0.00000000e+00],
        [0.00000000e+00, 1.23523911e-05, 0.00000000e+00],
        [0.00000000e+00, 1.22147268e-05, 0.00000000e+00],
        [0.00000000e+00, 1.14343557e-05, 0.00000000e+00],
        [0.00000000e+00, 1.04943153e-05, 0.00000000e+00],
        [0.00000000e+00, 1.04227588e-05, 0.00000000e+00],
        [0.00000000e+00, 1.01457947e-05, 0.00000000e+00],
        [0.00000000e+00, 9.71383493e-06, 0.00000000e+00],
        [0.00000000e+00, 9.20967069e-06, 0.00000000e+00],
        [0.00000000e+00, 8.90922222e-06, 0.00000000e+00],
        [0.00000000e+00, 8.85384805e-06, 0.00000000e+00],
        [0.00000000e+00, 8.67631104e-06, 0.00000000e+00],
        [0.00000000e+00, 8.63759884e-06, 0.00000000e+00],
        [0.00000000e+00, 8.62959079e-06, 0.00000000e+00],
        [0.00000000e+00, 8.26807851e-06, 0.00000000e+00],
        [0.00000000e+00, 8.00417152e-06, 0.00000000e+00],
        [0.00000000e+00, 8.00404622e-06, 0.00000000e+00],
        [0.000

In [ ]:
# Getting Betti Sequence
betti_sequence = BettiCurve()

m10_sample_betti_seq = betti_sequence.fit_transform(m10_sample_per_diag)
betti_sequence.fit_transform_plot(m10_sample_per_diag)

array([[[127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127,
         126, 126, 125, 124, 124, 124, 123, 122, 122, 122, 120, 118,
         118, 118, 118, 115, 111, 111, 109, 107, 102,  98,  97,  97,
          95,  94,  91,  86,  82,  78,  73,  64,  60,  59,  53,  51,
          50,  47,  46,  42,  40,  39,  37,  34,  32,  30,  25,  23,
          21,  20,  17,  15,  15,  14,  14,  14,  12,  11,   9,   9,
           9,   8,   8,   8,   8,   7,   7,   7,   6,   6,   5,   4,
           4,   4,   4,   4,   4,   4,   3,   3,   3,   3,   3,   3,
           2,   1,   1,   0],
        [  1,   0,   0,   1,   1,   3,   3,   3,   4,   3,   4,   4,
           3,   3,   3,   4,   4,   5,   7,  10,   9,   8,  10,  10,
          11,  11,  14,  14,  15,  15,  14,  14,  14,  14,  14,  14,
          12,   9,  10,  10,   9,   7,   8,   8,   8,   9,   8,   8,
           8,   7,   6,   7,   8,   6,   6,   4,   3,   3,   3,   3,
           3,   3,   4,   2,   3,   3,   3,   3,   3,   2,   1,   1,
    

In [ ]:
# Getting Betti Sequence
betti_sequence = BettiCurve()

m10_sample_betti_seq = betti_sequence.fit_transform(m10_sample_per_diag)
betti_sequence.fit_transform_plot(m10_sample_per_diag)

array([[[127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127, 127,
         126, 126, 125, 124, 124, 124, 123, 122, 122, 122, 120, 118,
         118, 118, 118, 115, 111, 111, 109, 107, 102,  98,  97,  97,
          95,  94,  91,  86,  82,  78,  73,  64,  60,  59,  53,  51,
          50,  47,  46,  42,  40,  39,  37,  34,  32,  30,  25,  23,
          21,  20,  17,  15,  15,  14,  14,  14,  12,  11,   9,   9,
           9,   8,   8,   8,   8,   7,   7,   7,   6,   6,   5,   4,
           4,   4,   4,   4,   4,   4,   3,   3,   3,   3,   3,   3,
           2,   1,   1,   0],
        [  1,   0,   0,   1,   3,   3,   3,   3,   4,   4,   4,   3,
           3,   3,   4,   4,   7,  10,   9,   9,  10,   9,  11,  12,
          14,  15,  15,  14,  14,  14,  14,  14,  13,  12,  10,  10,
           9,   7,   8,   8,   8,   9,   8,   8,   7,   6,   7,   8,
           6,   6,   4,   4,   3,   3,   3,   3,   4,   3,   3,   3,
           3,   3,   2,   1,   1,   1,   1,   1,   1,   1,   2,   2,
    

# TDA methods for all epochs of 19 EEG channels

In [ ]:
# Extract time series data
m10_all_ch_ts = epochs.get_data()
print(m10_all_ch_ts.shape)

n_samples = m10_all_ch_ts.shape[0]
n_channels = m10_all_ch_ts.shape[1]
n_times = m10_all_ch_ts.shape[2]

(467, 19, 400)


In [ ]:
# Build attractor
embedding_dimension = 3
embedding_time_delay = 8
stride = 3 # downsampling

attractor = TakensEmbedding(
#    n_jobs=2,
    time_delay=embedding_time_delay,
    dimension=embedding_dimension,
    stride=stride,
)

In [ ]:
m10_all_ch_point_cl = []

for i in range(n_samples): 
    m10_all_ch_point_cl.append(attractor.fit_transform(m10_all_ch_ts[i]))
m10_all_ch_point_cl = np.array(m10_all_ch_point_cl)

print(m10_all_ch_point_cl.shape)

attractor.fit_transform_plot(m10_all_ch_ts[10])

(467, 19, 128, 3)


array([[[-3.17079541e-06,  1.35600439e-06, -4.43188988e-06],
        [ 5.25846327e-07,  1.29812757e-06,  5.86063654e-07],
        [-3.47016852e-09,  4.96838163e-06,  5.11707004e-06],
        [-3.72900204e-06,  5.37844380e-06, -3.15447326e-07],
        [-4.98034332e-06,  6.70009511e-06,  2.71250618e-06],
        [-8.15358394e-07, -5.25960806e-07,  1.34015529e-07],
        [ 3.04291241e-06,  1.61422829e-06,  7.18581205e-06],
        [ 5.04796144e-06,  2.49263804e-06,  3.54399873e-06],
        [ 4.48565002e-06,  4.41748478e-06,  4.61848804e-07],
        [-3.40320497e-07,  2.03623245e-06,  4.66759543e-06],
        [ 2.92357525e-06,  3.61690053e-06, -2.20208000e-07],
        [ 1.34644629e-06,  1.87279086e-06,  1.29136316e-06],
        [ 1.49035102e-06,  3.05204349e-06, -3.00314050e-06],
        [ 4.13436941e-06, -2.42985833e-06, -8.08335740e-07],
        [-4.39011451e-07, -7.92032821e-07, -1.17294439e-06],
        [-4.76648502e-07, -2.56780712e-06,  1.63181793e-06],
        [-2.57076112e-06

In [ ]:
homology_dimensions = [0, 1, 2]

persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, 
    n_jobs=-1,
)

m10_all_ch_per_diag = []

for i in range(n_samples):
    m10_all_ch_per_diag.append(persistence.fit_transform(m10_all_ch_point_cl[i]))


In [ ]:
print(m10_all_ch_per_diag[0].shape)
print(m10_all_ch_per_diag[1].shape)
print(m10_all_ch_per_diag[2].shape)

#m10_all_ch_per_diag = np.array(m10_all_ch_per_diag)

print("Persistence diagram for periodic signal")
persistence.fit_transform_plot(m10_all_ch_point_cl[10])

(19, 186, 3)
(19, 191, 3)
(19, 189, 3)
Persistence diagram for periodic signal


array([[[0.00000000e+00, 2.12445300e-07, 0.00000000e+00],
        [0.00000000e+00, 3.56654539e-07, 0.00000000e+00],
        [0.00000000e+00, 5.00457134e-07, 0.00000000e+00],
        [0.00000000e+00, 6.56809391e-07, 0.00000000e+00],
        [0.00000000e+00, 6.66324354e-07, 0.00000000e+00],
        [0.00000000e+00, 7.53537392e-07, 0.00000000e+00],
        [0.00000000e+00, 7.53945812e-07, 0.00000000e+00],
        [0.00000000e+00, 7.56780992e-07, 0.00000000e+00],
        [0.00000000e+00, 8.23580933e-07, 0.00000000e+00],
        [0.00000000e+00, 8.37417701e-07, 0.00000000e+00],
        [0.00000000e+00, 8.43003647e-07, 0.00000000e+00],
        [0.00000000e+00, 8.51227810e-07, 0.00000000e+00],
        [0.00000000e+00, 8.58795829e-07, 0.00000000e+00],
        [0.00000000e+00, 8.63471939e-07, 0.00000000e+00],
        [0.00000000e+00, 8.77107027e-07, 0.00000000e+00],
        [0.00000000e+00, 8.91057255e-07, 0.00000000e+00],
        [0.00000000e+00, 9.06281628e-07, 0.00000000e+00],
        [0.000

In [ ]:
# Getting Betti Sequence
betti_sequence = BettiCurve()

m10_all_ch_betti_seq = []

for i in range(n_samples):
    m10_all_ch_betti_seq.append(betti_sequence.fit_transform(m10_all_ch_per_diag[i]))

m10_all_ch_betti_seq = np.array(m10_all_ch_betti_seq)

print(m10_all_ch_betti_seq.shape)

betti_sequence.fit_transform_plot(m10_all_ch_per_diag[50])

(467, 19, 3, 100)


array([[[127, 127, 126, 120, 117, 109,  98,  88,  77,  65,  58,  47,
          36,  31,  22,  18,  17,  15,  13,  13,  12,   9,   6,   6,
           6,   5,   5,   5,   4,   3,   3,   1,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0],
        [  0,   0,   1,   1,   1,   5,   4,   5,   6,   6,   8,   3,
           6,   7,   7,   1,   0,   0,   1,   1,   1,   1,   0,   0,
           0,   1,   2,   2,   2,   2,   2,   2,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
    

# Computing Persistence Entropies

In [ ]:
from gtda.diagrams import PersistenceEntropy

In [ ]:
# Not normalized

PE = PersistenceEntropy()

m10_all_ch_PE_not_norm = []

for i in range(n_samples):
    m10_all_ch_PE_not_norm.append(PE.fit_transform(m10_all_ch_per_diag[i]))

m10_all_ch_PE_not_norm = np.array(m10_all_ch_PE_not_norm)

print(m10_all_ch_PE_not_norm.shape)

(467, 19, 3)


In [ ]:
# Normalized

PE = PersistenceEntropy(normalize=True)

m10_all_ch_PE_norm = []

for i in range(n_samples):
    m10_all_ch_PE_norm.append(PE.fit_transform(m10_all_ch_per_diag[i]))

m10_all_ch_PE_norm = np.array(m10_all_ch_PE_norm)

print(m10_all_ch_PE_norm.shape)

(467, 19, 3)
